In [62]:
import pickle as p
import factorgraph as fg
def BP_with_penalty(queryPath_2gram):
    global LBP_MAX_ITERS
    global THRES
    threshold=THRES
    g = fg.Graph()
    global_node_dict = dict()
    global_edge_dict = dict()
    rvlabels = defaultdict(list)
    if type(queryPath_2gram) == type(''):
        query_2gram = p.load(open(queryPath_2gram, 'rb'))
    else:
        query_2gram = queryPath_2gram
    hasDistance = False
    if type(query_2gram[0][0][0]) == type((1,2)):
        hasDistance = True
        
    for i in query_2gram:
        if i[2] > threshold:
            if not hasDistance:
                test_src = i[0][0]
                test_des = i[0][1]
            else:
                test_src = i[0][0][0]
                test_des = i[0][0][1]
            if test_src == test_des:
                continue
            # initialize the dictionary
            if not global_node_dict.has_key(test_src):
                global_node_dict[test_src] = []
            if not global_node_dict.has_key(test_des):
                global_node_dict[test_des] = []
            if not global_edge_dict.has_key((test_src, test_des)):
                global_edge_dict[(test_src, test_des)] = []
            
            
            funcList = i[1]
            for predicted_func in funcList:
                if hasDistance:
                    querydistance = i[0][1]
                    resultdistance = predicted_func[3]
                    if querydistance != resultdistance:
                        continue

                src_funcname = predicted_func[2][0]
                des_funcname = predicted_func[2][1]
                libraryname = predicted_func[0]

                # find the src function if it's already in the dict,
                # otherwise, add it to the dict
                if (src_funcname, libraryname) not in global_node_dict[test_src]:
                    global_node_dict[test_src].append((src_funcname, libraryname))
                    rvlabels[test_src].append(src_funcname + ', ' + '(' + src_funcname + ', ' + des_funcname + ')')
                # do the same thing to des function
                if (des_funcname, libraryname) not in global_node_dict[test_des]:
                    global_node_dict[test_des].append((des_funcname, libraryname))
                    rvlabels[test_des].append(des_funcname + ', ' + '(' + src_funcname + ', ' + des_funcname + ')')
                
                # add this edge to the edge dictionary
                if (src_funcname, des_funcname, libraryname) not in global_edge_dict[(test_src, test_des)]:
                    global_edge_dict[(test_src, test_des)].append((src_funcname, des_funcname, libraryname))
    
    # add functions as random variables
    for key in global_node_dict:
        #src_list = global_dict[key]
        #a = [list(i) for i in src_list]
        #if key not in [row[0] for row in a]:
        #global_dict[key] = list(set((item[0], item[1]) for item in global_dict[key]))
        global_node_dict[key].append(('None', 'None'))
        rvlabels[test_src].append('None')
        g.rv(key, len(global_node_dict[key])) # None label is the last item
    p.dump(global_node_dict, open('global_node_dict','w'))
    
    # add factors
    for (test_src, test_des) in global_edge_dict.keys():
        src_rv = g.get_rvs()[test_src]
        des_rv = g.get_rvs()[test_des]
        src_list = global_node_dict[test_src]
        des_list = global_node_dict[test_des]
        # set the probability for src_rv and des_rv
        # first find if there is an existing probability, if so, add to it
        # otherwise, initialize a new probability matrix
        factor1 = None
        probability1 = None

        if test_src != test_des:
            pot=np.full((len(src_list), len(des_list)), 0.1)
            pot[-1, -1] = 0.2
            factor1 = g.factor([src_rv, des_rv], potential=pot, name = '')
            probability1 = factor1.get_potential()
            
        # if there is an matching edge
        if len(global_edge_dict[(test_src, test_des)]) > 0:
            for (src_funcname, des_funcname, libraryname) in global_edge_dict[(test_src, test_des)]:
                src_ind = src_list.index((src_funcname, libraryname))
                des_ind = des_list.index((des_funcname, libraryname))
                if src_rv == factor1.get_rvs()[0]:
                    probability1[src_ind, des_ind] = 0.99
                else:
                    probability1[des_ind, src_ind] = 0.99
                    
        else:
            if len(src_list) > 0 and len(des_list) > 0:
                probability1[:, -1] = 0.3
                probability1[-1, :] = 0.3
#             elif len(src_list) > 0 and len(des_list) == 0:
#                 if src_rv == factor1.get_rvs()[0]:
#                     probability1[-1, 0] = 0.3
#                 else:
#                     probability1[0, -1] = 0.3
#             elif len(src_list) == 0 and len(des_list) > 0:
#                 if src_rv == factor1.get_rvs()[0]:
#                     probability1[0, -1] = 0.3
#                 else:
#                     probability1[-1, 0] = 0.3
            else:
                continue
        factor1.set_potential(probability1)
        #factor1.name += src_funcname + '->' + des_funcname + '\n'
        
    iters, converged = g.lbp(global_node_dict, normalize=True, max_iters=LBP_MAX_ITERS, progress=False)
    g.global_dict = global_node_dict
    print 'LBP ran for %d iterations. Converged = %r' % (iters, converged)
    results = g.get_func_prediction(normalize=True)
    p.dump(results, open('prediction','w'))
    results = g.accuracy_of_functions(normalize=True)
    return g

In [63]:
global LBP_MAX_ITERS
global THRES
LBP_MAX_ITERS =40
THRES = 0.9
knn = p.load(open('/home/yijiufly/Downloads/codesearch/data/versiondetect/test3/nginx/nginx-{openssl-1.0.1d}{zlib-1.2.11}/test_kNN_1009_2gram.p', 'r'))

In [64]:
from collections import defaultdict
import numpy as np
g = BP_with_penalty(knn)

LBP ran for 40 iterations. Converged = 0.9263136497611546
correct prediction, None prediction, total prediction, precision:
1538 2001 4142 0.718355908454


In [69]:
results = g.get_func_prediction(normalize=True)
#print results
none_nodes = set()
for key in results.keys():
    prediction = list(results[key])
    if len(prediction)==1 and prediction[0][0][0] == 'None':
        none_nodes.add(key)
p.dump(none_nodes, open('none_nodes', 'w'))

In [68]:
g.print_graph('BP.dot')

In [67]:
global_node_dict = p.load(open('global_node_dict','r'))
wrong_nodes = set()
for key in global_node_dict.keys():
    predictions = [i[0] for i in global_node_dict[key]]
    if key not in predictions:
        wrong_nodes.add(key)
print wrong_nodes
p.dump(wrong_nodes, open('wrong_nodes', 'w'))
results = g.get_func_prediction(normalize=True)
#print results
wrong_predictions = set()
for key in results.keys():
    predictions = [i[0][0] for i in results[key]]
    if key in predictions:
        continue
    elif len(predictions) == 1 and predictions[0] == 'None':
        continue
    else:
        wrong_predictions.add(key)
p.dump(wrong_predictions, open('wrong_predictions', 'w'))

set(['ngx_http_parse_chunked', 'ngx_http_proxy_pass.part.4', 'ngx_http_add_cache_control', 'X509_VERIFY_PARAM_table_cleanup', 'ngx_mail_auth_http_read_handler', 'pkey_gost2001_derive', 'ngx_http_copy_thread_handler', 'ngx_mail_pop3_create_srv_conf', 'print_qualifiers', 'int_thread_release', 'ngx_http_upstream_copy_last_modified', 'ngx_destroy_pool', 'ngx_http_autoindex_create_loc_conf', 'SSL_CTX_set_client_cert_engine', 'ngx_open_and_stat_file', 'emit_mov_byte', 'SXNET_add_id_INTEGER', 'EVP_PKEY_assign', 'gost_enc_cfb', 'ASN1_template_new', 'ngx_resolver_cleanup', 'DES_ede3_cfb64_encrypt', 'ngx_http_mp4_read_stss_atom', 'SSL_SESSION_new', 'ngx_http_fastcgi_create_request', 'ssl3_record_sequence_update', 'ngx_http_uwsgi_cache_key', 'ASN1_d2i_fp', 'ngx_http_validate_host', 'ngx_http_file_cache_valid_set_slot', 'ngx_stream_log_error', 'ngx_http_block_reading', 'ASN1_UTCTIME_set', 'BN_mod_lshift1', 'rc2_cfb64_cipher', 'OBJ_cleanup', 'ngx_create_temp_buf', 'ngx_http_rewrite_if', 'ngx_array_

In [66]:
results = g.get_func_prediction(normalize=True)
print results

defaultdict(<type 'set'>, {'ngx_http_parse_chunked': set([(('PKCS5_PBKDF2_HMAC', 'libcrypto'), 0.289249536518956)]), 'SSL_srp_server_param_with_username': set([(('None', 'None'), 0.20068747813096002)]), 'PKCS7_simple_smimecap': set([(('PKCS7_simple_smimecap', 'libcrypto'), 0.9889410867355443)]), 'X509_PURPOSE_get_count': set([(('X509_PURPOSE_get_count', 'libcrypto'), 0.4759615384615385), (('X509_VERIFY_PARAM_get_count', 'libcrypto'), 0.4759615384615385)]), 'ngx_http_add_cache_control': set([(('None', 'None'), 1.0)]), 'RSA_padding_check_PKCS1_type_2': set([(('RSA_padding_check_PKCS1_type_2', 'libcrypto'), 0.8249305614005537)]), 'ngx_http_upstream_process_header': set([(('None', 'None'), 0.6218777330853389)]), 'RSA_padding_check_PKCS1_type_1': set([(('RSA_padding_check_PKCS1_type_1', 'libcrypto'), 0.8249305613993143)]), 'ngx_mail_auth_http_read_handler': set([(('None', 'None'), 1.0)]), 'PEM_read_bio_Parameters': set([(('PEM_read_bio_Parameters', 'libcrypto'), 0.9949443471189009)]), 'ngx_

In [72]:
path = '/home/yijiufly/Downloads/codesearch/data/versiondetect/test3/nginx/nginx-{openssl-1.0.1k}{zlib-1.2.11}/nginx-{openssl-1.0.1k}{zlib-1.2.11}.ida_newmodel.emb'
embd = p.load(open(path, 'r'))
print len(embd[0])

64
